# Model selection

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [3]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [4]:
import seaborn as sns

Import data from csv files into a dataframe

In [5]:
data = pd.read_csv('Combined_all_3rd_fit.csv', encoding='cp1252')
print(data.columns.values)

['Name' 'Coef_a' 'Coef_b' 'Coef_c' 'Coef_d' 'A_site' 'B_site' 'X_site'
 'Spacegroup' 'Ehull' 'BulkModulus' 'Energy' 'ZPE' 's_A' 's_B' 's_X'
 'density' 'mean_A2B' 'mean_A2X' 'mean_B2X' 'mean_X2X' 'std_A2B' 'std_A2X'
 'std_B2X' 'std_X2X' 'E_coh' 'TF' 'OF' 'A_Z' 'B_Z' 'X_Z' 'A_M' 'B_M' 'X_M'
 'A_G' 'B_G' 'X_G' 'A_IEI' 'B_IEI' 'X_IEI' 'A_IEII' 'B_IEII' 'X_IEII'
 'A_EA' 'B_EA' 'X_EA' 'A_ChiP' 'B_ChiP' 'X_ChiP' 'A_ChiA' 'X_ChiA'
 'A_Rvdw' 'B_Rvdw' 'X_Rvdw' 'A_Rc' 'B_Rc' 'X_Rc' 'A_Ra' 'B_Ra' 'X_Ra'
 'A_MP' 'B_MP' 'X_MP' 'A_BP' 'B_BP' 'X_BP' 'A_Rho' 'B_Rho' 'A_MV' 'B_MV'
 'X_MV' 'A_Hf' 'B_Hf' 'X_Hf' 'A_Hv' 'B_Hv' 'X_Hv' 'A_Kappa' 'B_Kappa'
 'X_Kappa' 'A_CvM' 'B_CvM' 'X_CvM' 'A_B' 'B_B' 'X_B' 'A_MendeleevNo'
 'B_MendeleevNo' 'X_MendeleevNo']


Drop the unnecessary columns and fill the empty cells with zero

In [6]:
data.drop(['Name', 'A_site', 'B_site', 'X_site', 'Spacegroup','BulkModulus',
           'Ehull','Energy','ZPE','Coef_b', 'Coef_c', 'Coef_a'], axis=1, inplace = True)
data.fillna(0, inplace= True)
d=data.copy()
columns = list(d.columns.values)
print(d.shape)

(80, 77)


Set the target variable to machine learn

In [7]:
target = 'Coef_d'

Define various methods to be used for buildig and validating the models

In [8]:
"""
    Functions to perform scaling
    
    """
def standard_scaling(target):
    data_std=d.copy()
    data_std[columns]= StandardScaler().fit_transform(d[columns])
    Y = data_std[target] 
    X = data_std.drop([target], axis=1)
    return X,Y

def minmax_scaling(target):
    data_mm=d.copy()
    data_mm[columns]= MinMaxScaler().fit_transform(d[columns])
    Y = data_mm[target] 
    X = data_mm.drop([target], axis=1)
    return X,Y

In [9]:
"""
    Function to build machine learning models by hyper parameter tuning. It also plots scatter plot
    
    """
def build_cv_model(X,Y,b_drop=False,target='Coef_d'):
    if b_drop :
        correlated_features = ["A_Rc","A_Ra","A_M","A_MP","A_MV","A_MendeleevNo","A_Hf","A_Hv","B_Rc",
                       "B_Rvdw","B_M","B_BP","B_MendeleevNo","B_Hv","X_Rc","X_Rvdw","X_M","X_BP","X_MP",
                       "X_MendeleevNo","X_Hf","X_Hv","X_G","X_B","X_CvM","X_ChiP"]
        X.drop(labels=correlated_features, axis=1, inplace=True)
        
    print("The shape of X is ",X.shape)
    
    ml_model, cv_results = train_model(X,Y,hyperparams=hyperparams, cv=True, return_cv=True)
    
    return ml_model

In [10]:
def run_model(X,Y,model,target='Coef_d'):
    rmse,r2 = run_cv(model, X, Y, n_cv = 5)
    print("-----------------------------------------------------")
    print("The R2 is ",r2)
    print("The RMSE is ",rmse)

In [11]:
"""
    Function to perform feature elimination using select K best of RFE method. It also prints the 
    feature scores obtained as:
    SelectKBest: The scores obtained from the selector
    RFE: The feature importance obtained from the RFR model 
    
    """
def feature_elimination(X,Y,n,estimator,method='skb'):
    if method=='skb':
        bestfeatures = SelectKBest(score_func=f_regression, k=10)
        fit = bestfeatures.fit(X,Y)
        dfscores = pd.DataFrame(fit.scores_)
        dfcolumns = pd.DataFrame(X.columns)
        #concat two dataframes for better visualization 
        featureScores = pd.concat([dfcolumns,dfscores],axis=1)
        featureScores.columns = ['Feature','Score']  #naming the dataframe columns
        X=X[featureScores.nlargest(n,'Score')['Feature'].values]
    elif method=='rfe':
        selector = RFE(estimator, n_features_to_select=n, step=1)
        selector = selector.fit(X, Y)
        dfscores = pd.DataFrame(selector.ranking_)
        dfcolumns = pd.DataFrame(selector.feature_names_in_)
        #concat two dataframes for better visualization 
        featureSelection = pd.concat([dfcolumns,dfscores],axis=1)
        featureSelection.columns = ['Feature','Score']  #naming the dataframe columns
        X=X[featureSelection.nsmallest(n,'Score')['Feature'].values]
        estimator.fit(X,Y)
        importance = pd.DataFrame(estimator.feature_importances_)
        featureNames = pd.DataFrame(X.columns.values)
        featureScores = pd.concat([featureNames,importance],axis=1)
        featureScores.columns = ['Feature','Score']  #naming the dataframe columns
        
    print(featureScores)
    print("-----------------------------------------------------")
    return X

In [12]:
def train_model(X, y, hyperparams = None, cv = False, return_cv = False):
    """
    Function to train the ML model on the given data X (features) and y (target property). 
    If hyperparams argument is passed, all possible combinations of alpha and kernel values
    will be tried to find combination with minimum CV error. The final  model is trained
    using the fixed kernel and alpha params as determined using CV.
    
    """
    
    if cv:
        cv_results = {'alpha':[],'cv_rmse':[],'cv_r2':[]}
        for a in hyperparams['alpha']:

            lasso = Lasso(alpha=a)

            cv_error, cv_r2 = run_cv(lasso, X, y, n_cv = 5)
            cv_results['cv_rmse'].append(cv_error)
            cv_results['cv_r2'].append(cv_r2)
            cv_results['alpha'].append(a)

        cv_results = pd.DataFrame(cv_results)
        cv_results = cv_results.sort_values('cv_rmse')
        a_opt = cv_results.iloc[0]['alpha']
        
    else:
        a_opt = hyperparams['alpha'][0]

    lasso = Lasso(alpha=a_opt)
    model = lasso.fit(X,y)
    print("-----------------------------------------------------")
    print("The R2 for ",a_opt," is ",cv_results.iloc[0]['cv_r2'])
    print("The RMSE for ",a_opt," is ",cv_results.iloc[0]['cv_rmse'])
    return [model, cv_results] if return_cv else model


def run_cv(ml_model, X, y, n_cv = 5):
    """
    Function to run Cross-validation
    """
    kf = KFold(n_splits=n_cv,shuffle=True,random_state=50)
    y_val = []
    y_pred = []

    for idx, (train, val) in enumerate(kf.split(X)):
        
        X_cv_train = X.values[train]
        X_cv_val = X.values[val]

        y_cv_train = y.values[train]
        y_cv_val = y.values[val]    

        # Model fit and prediction
        model = ml_model.fit(X_cv_train, y_cv_train)
        y_pred_val = model.predict(X_cv_val)
        
        y_val.append(y_cv_val)
        y_pred.append(y_pred_val)

    # Computing errors
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = r2_score(y_val, y_pred)
    
    return rmse, r2

Define the hyperparameters to be tuned for the given algorithm

In [13]:
hyperparams={'alpha': [0.0001,0.0005,0.001,0.005,0.01,0.02,0.05,0.1,1,2,4,8]}

## Without scaling

In [14]:
data_std=d.copy()
Y = data_std[target] 
X = data_std.drop([target], axis=1)

In [15]:
ml_model=build_cv_model(X,Y,False,target)

The shape of X is  (80, 76)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.189e+01, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.175e+01, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  2.0  is  0.7809445553631749
The RMSE for  2.0  is  2.8130023470842374


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.529e+01, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.359e+01, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

### Drop correlated features

In [16]:
ml_model_prime=build_cv_model(X,Y,True,target)

/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.034e+01, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.958e+01, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

The shape of X is  (80, 50)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.324e+01, tolerance: 6.213e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.526e+01, tolerance: 7.884e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.1  is  0.7771946806957974
The RMSE for  0.1  is  3.1684067952639765


### Feature elimination

In [17]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0        s_A  29.880346
1        s_B  61.964962
2        s_X  43.799906
3    density   1.191452
4   mean_A2B  47.874859
5   mean_A2X  22.081705
6   mean_B2X   0.045034
7   mean_X2X  33.902649
8    std_A2B   7.106288
9    std_A2X  12.236469
10   std_B2X  16.266083
11   std_X2X   5.427746
12     E_coh  11.792747
13        TF   7.008817
14        OF  19.424616
15       A_Z   5.970007
16       B_Z  20.419989
17       X_Z  57.414810
18       A_G   3.457040
19       B_G   0.902234
20     A_IEI  23.950524
21     B_IEI   0.055573
22     X_IEI   0.848306
23    A_IEII   0.385163
24    B_IEII   2.519527
25    X_IEII  55.097923
26      A_EA  12.192466
27      B_EA   1.411509
28      X_EA  95.250147
29    A_ChiP  11.872273
30    B_ChiP   0.622186
31    A_ChiA  15.842395
32    X_ChiA   0.005879
33    A_Rvdw  27.535795
34      B_Ra  24.230307
35      X_Ra  67.173814
36      B_MP  10.539184
37      A_BP   8.076791
38     A_Rho   0.830238
39     B_Rho   8.929870
40      B_MV   7

/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.452e+01, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.563e+01, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

In [18]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
30       A_Z   5.970007
31   A_Kappa   5.590909
32   std_X2X   5.427746
33     A_CvM   3.585042
34       A_B   3.534708
35       A_G   3.457040
36    B_IEII   2.519527
37   X_Kappa   2.297719
38       B_B   1.964270
39   B_Kappa   1.742806
----------------

/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.122e+02, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.188e+01, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

In [19]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
-----------------------------------------------------
The shape of X is  (80, 20)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.230e+02, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.141e+02, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.05  is  0.7110902004515445
The RMSE for  0.05  is  2.9642415558179915
-----------------------------------------------------
The R2 is  0.714115472874196
The RMSE is  3.0471211962764264


In [20]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
-----------------------------------------------------
The shape of X is  (80, 10)
-----------------------------------------------------
The R2 for  0.05  is  0.734232637021039
The RMSE for  0.05  is  3.425246182085314
-----------------------------------------------------
The R2 is  0.7268825815150584
The RMSE is  3.4937511857606975


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.439e+01, tolerance: 7.327e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.376e+02, tolerance: 8.288e-01
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

## Standard scaling

In [21]:
X,Y = standard_scaling(target)
ml_model=build_cv_model(X,Y,False,target)

The shape of X is  (80, 76)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.079e-01, tolerance: 6.338e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.046e-02, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.01  is  0.7893012652375363
The RMSE for  0.01  is  0.2488895547733669


### Drop correlated features

In [22]:
ml_model_prime=build_cv_model(X,Y,True,target)

The shape of X is  (80, 50)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.200e-01, tolerance: 6.338e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.599e-01, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.02  is  0.7970598891344353
The RMSE for  0.02  is  0.2562040981234409


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.136e-02, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.756e-03, tolerance: 6.260e-03
  model = cd_fast.enet_coordinate_descent(


### Feature elimination

In [23]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0        s_A  29.880346
1        s_B  61.964962
2        s_X  43.799906
3    density   1.191452
4   mean_A2B  47.874859
5   mean_A2X  22.081705
6   mean_B2X   0.045034
7   mean_X2X  33.902649
8    std_A2B   7.106288
9    std_A2X  12.236469
10   std_B2X  16.266083
11   std_X2X   5.427746
12     E_coh  11.792747
13        TF   7.008817
14        OF  19.424616
15       A_Z   5.970007
16       B_Z  20.419989
17       X_Z  57.414810
18       A_G   3.457040
19       B_G   0.902234
20     A_IEI  23.950524
21     B_IEI   0.055573
22     X_IEI   0.848306
23    A_IEII   0.385163
24    B_IEII   2.519527
25    X_IEII  55.097923
26      A_EA  12.192466
27      B_EA   1.411509
28      X_EA  95.250147
29    A_ChiP  11.872273
30    B_ChiP   0.622186
31    A_ChiA  15.842395
32    X_ChiA   0.005879
33    A_Rvdw  27.535795
34      B_Ra  24.230307
35      X_Ra  67.173814
36      B_MP  10.539184
37      A_BP   8.076791
38     A_Rho   0.830238
39     B_Rho   8.929870
40      B_MV   7

/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.742e-01, tolerance: 6.338e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.842e-01, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.02  is  0.8012111973063559
The RMSE for  0.02  is  0.25432712973984384
-----------------------------------------------------
The R2 is  0.8012111973063559
The RMSE is  0.25432712973984384


In [24]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.334e-01, tolerance: 6.338e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.803e-02, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
30       A_Z   5.970007
31   A_Kappa   5.590909
32   std_X2X   5.427746
33     A_CvM   3.585042
34       A_B   3.534708
35       A_G   3.457040
36    B_IEII   2.519527
37   X_Kappa   2.297719
38       B_B   1.964270
39   B_Kappa   1.742806
----------------

In [25]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
-----------------------------------------------------
The shape of X is  (80, 20)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.307e-02, tolerance: 6.338e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.829e-02, tolerance: 7.169e-03
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.02  is  0.7214685679639559
The RMSE for  0.02  is  0.26046331674866585
-----------------------------------------------------
The R2 is  0.7214685679639559
The RMSE is  0.26046331674866585


In [26]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
-----------------------------------------------------
The shape of X is  (80, 10)
-----------------------------------------------------
The R2 for  0.01  is  0.7302509099183845
The RMSE for  0.01  is  0.3184241725034789
-----------------------------------------------------
The R2 is  0.713595073212228
The RMSE is  0.3219430471367615


## Minmax scaling

In [27]:
X,Y = minmax_scaling(target)
ml_model=build_cv_model(X,Y,False,target)

The shape of X is  (80, 76)


/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.374e-04, tolerance: 3.397e-04
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.674e-04, tolerance: 3.842e-04
  model = cd_fast.enet_coordinate_descent(
/Users/krishnarajmayya/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

-----------------------------------------------------
The R2 for  0.001  is  0.7684860786162645
The RMSE for  0.001  is  0.056957480961694264


### Drop correlated features

In [28]:
ml_model_prime=build_cv_model(X,Y,True,target)

The shape of X is  (80, 50)
-----------------------------------------------------
The R2 for  0.001  is  0.7764410806027999
The RMSE for  0.001  is  0.06179839155463226


### Feature elimination

In [29]:
X=feature_elimination(X,Y,40,ml_model_prime,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0        s_A  29.880346
1        s_B  61.964962
2        s_X  43.799906
3    density   1.191452
4   mean_A2B  47.874859
5   mean_A2X  22.081705
6   mean_B2X   0.045034
7   mean_X2X  33.902649
8    std_A2B   7.106288
9    std_A2X  12.236469
10   std_B2X  16.266083
11   std_X2X   5.427746
12     E_coh  11.792747
13        TF   7.008817
14        OF  19.424616
15       A_Z   5.970007
16       B_Z  20.419989
17       X_Z  57.414810
18       A_G   3.457040
19       B_G   0.902234
20     A_IEI  23.950524
21     B_IEI   0.055573
22     X_IEI   0.848306
23    A_IEII   0.385163
24    B_IEII   2.519527
25    X_IEII  55.097923
26      A_EA  12.192466
27      B_EA   1.411509
28      X_EA  95.250147
29    A_ChiP  11.872273
30    B_ChiP   0.622186
31    A_ChiA  15.842395
32    X_ChiA   0.005879
33    A_Rvdw  27.535795
34      B_Ra  24.230307
35      X_Ra  67.173814
36      B_MP  10.539184
37      A_BP   8.076791
38     A_Rho   0.830238
39     B_Rho   8.929870
40      B_MV   7

In [30]:
X=feature_elimination(X,Y,30,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
30       A_Z   5.970007
31   A_Kappa   5.590909
32   std_X2X   5.427746
33     A_CvM   3.585042
34       A_B   3.534708
35       A_G   3.457040
36    B_IEII   2.519527
37   X_Kappa   2.297719
38       B_B   1.964270
39   B_Kappa   1.742806
----------------

In [31]:
X=feature_elimination(X,Y,20,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
20    A_ChiP  11.872273
21     E_coh  11.792747
22      B_MP  10.539184
23     B_CvM  10.158055
24     B_Rho   8.929870
25      A_BP   8.076791
26      X_MV   7.709740
27      B_MV   7.166166
28   std_A2B   7.106288
29        TF   7.008817
-----------------------------------------------------
The shape of X is  (80, 20)
-----------------------------------------------------
The R2 for  0.001  is  0.7324814587145096
The RMSE for  0.001  is  0.06032077505479283
---------------------------------

In [32]:
X=feature_elimination(X,Y,10,ml_model,method='skb')
ml_model=build_cv_model(X,Y,False,target)
run_model(X,Y,ml_model_prime,target)

     Feature      Score
0       X_EA  95.250147
1       X_Ra  67.173814
2        s_B  61.964962
3        X_Z  57.414810
4     X_IEII  55.097923
5   mean_A2B  47.874859
6        s_X  43.799906
7   mean_X2X  33.902649
8        s_A  29.880346
9     A_Rvdw  27.535795
10      B_Ra  24.230307
11     A_IEI  23.950524
12  mean_A2X  22.081705
13      B_Hf  21.205026
14       B_Z  20.419989
15        OF  19.424616
16   std_B2X  16.266083
17    A_ChiA  15.842395
18   std_A2X  12.236469
19      A_EA  12.192466
-----------------------------------------------------
The shape of X is  (80, 10)
-----------------------------------------------------
The R2 for  0.0005  is  0.7295212636381091
The RMSE for  0.0005  is  0.07377668051721521
-----------------------------------------------------
The R2 is  0.7145017871293675
The RMSE is  0.07449920730262055
